In [1]:
import numpy as np
import faiss
import litellm
from litellm import completion
from dotenv import load_dotenv
import os
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import ArxivLoader
from langchain_community.vectorstores import FAISS
from langchain_openai import OpenAIEmbeddings
from operator import itemgetter
from langchain.schema.output_parser import StrOutputParser
from langchain.schema.runnable import RunnablePassthrough
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI

# Load API keys
load_dotenv(".env")
apikey = os.getenv('OPENAI_API_KEY')

In [2]:
import requests

# Example query to the arXiv API directly
url = "http://export.arxiv.org/api/query"
params = {
    "search_query": "all:Retrieval Augmented Generation",
    "start": 0,
    "max_results": 5
}

response = requests.get(url, params=params)

if response.status_code == 200:
    print("API Request Successful")
    print("Response:", response.text[:500])  # Print first 500 characters of the response
else:
    print("Failed to fetch data from arXiv:", response.status_code)

API Request Successful
Response: <?xml version="1.0" encoding="UTF-8"?>
<feed xmlns="http://www.w3.org/2005/Atom">
  <link href="http://arxiv.org/api/query?search_query%3Dall%3ARetrieval%20Augmented%20Generation%26id_list%3D%26start%3D0%26max_results%3D5" rel="self" type="application/atom+xml"/>
  <title type="html">ArXiv Query: search_query=all:Retrieval Augmented Generation&amp;id_list=&amp;start=0&amp;max_results=5</title>
  <id>http://arxiv.org/api/UOKvU43HASfzk48jVrFyBsmll1A</id>
  <updated>2024-05-24T00:00:00-04:00</updat


In [3]:
# Load the document pertaining to a particular topic
docs = ArxivLoader(query=""" all:"attention mechanisms" AND (all:"convolutional neural networks" OR all:"CNN") AND NOT all:"transformer" """, load_max_docs=5).load()

# Split the dpocument into smaller chunks
text_splitter = RecursiveCharacterTextSplitter.from_tiktoken_encoder(
    chunk_size=350, chunk_overlap=50
)

chunked_documents = text_splitter.split_documents(docs)

In [4]:
print(type(docs))
print(len(docs))
print(type(docs[0].page_content))

print(type(chunked_documents))
print(len(chunked_documents))
print(chunked_documents[0])




<class 'list'>
5
<class 'str'>
<class 'list'>
199
page_content='1\nPulmonary Disease Classiﬁcation Using Globally\nCorrelated Maximum Likelihood:\nan Auxiliary Attention mechanism for\nConvolutional Neural Networks\nEdward Verenich, Tobias Martin, Alvaro Velasquez, Nazar Khan, and Faraz Hussain\nAbstract—Convolutional neural networks (CNN) are now being\nwidely used for classiﬁying and detecting pulmonary abnormal-\nities in chest radiographs. Two complementary generalization\nproperties of CNNs, translation invariance and equivariance,\nare particularly useful in detecting manifested abnormalities\nassociated with pulmonary disease, regardless of their spatial\nlocations within the image. However, these properties also come\nwith the loss of exact spatial information and global relative\npositions of abnormalities detected in local regions. Global\nrelative positions of such abnormalities may help distinguish\nsimilar conditions, such as COVID-19 and viral pneumonia. In\nsuch instance

In [5]:
# Test
response = completion(
    api_key=apikey,
    base_url="https://drchat.xyz",
    model = "gpt-3.5-turbo-16k",
    custom_llm_provider="openai",
    messages = [{ "content": "What is a cat?","role": "user"}],
    temperature=0.5
)
print(response.choices[0].message.content)


Give Feedback / Get Help: https://github.com/BerriAI/litellm/issues/new
LiteLLM.Info: If you need to debug this error, use `litellm.set_verbose=True'.



APIError: OpenAIException - Traceback (most recent call last):
  File "C:\Users\adity_724nfxg\AppData\Roaming\Python\Python311\site-packages\httpx\_transports\default.py", line 67, in map_httpcore_exceptions
    yield
  File "C:\Users\adity_724nfxg\AppData\Roaming\Python\Python311\site-packages\httpx\_transports\default.py", line 231, in handle_request
    resp = self._pool.handle_request(req)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\Users\adity_724nfxg\AppData\Roaming\Python\Python311\site-packages\httpcore\_sync\connection_pool.py", line 216, in handle_request
    raise exc from None
  File "C:\Users\adity_724nfxg\AppData\Roaming\Python\Python311\site-packages\httpcore\_sync\connection_pool.py", line 196, in handle_request
    response = connection.handle_request(
               ^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\Users\adity_724nfxg\AppData\Roaming\Python\Python311\site-packages\httpcore\_sync\connection.py", line 99, in handle_request
    raise exc
  File "C:\Users\ad

In [ ]:
user_query = input("Ask a research question!")

# Create multiple search queries
search_split_prompt = f"""
Your role is that of a researcher attempting to answer a question. Given a question from the user,
your job is to come up with an ArXiv query that searches for the exact information needed to answer the question.

You can include all syntax that involves including multiple terms, search by abstract, title, etc.

Example 1:
Given question: What are the ethical concerns associated with the use of facial recognition technology?
Your Answer: ("facial recognition technology" OR "facial recognition systems" OR "facial recognition software") AND ("ethical concerns" OR "ethical implications" OR "ethical issues")

Example 2:
Given question: What are some prominent attention mechanisms for convolutional neural networks, and how are they used in the autonomous vehicle industry?
Your Answer: all:"attention mechanisms" AND ("convolutional neural networks" OR "CNN") AND all:"attention mechanisms" AND ("autonomous vehicles" OR "self-driving cars")

Question: {user_query},

As shown above, your response should solely be an ArXiv Query, and nothing else.

"""
response = completion(
    api_key=apikey,
    base_url="https://drchat.xyz",
    model = "gpt4-1106-preview",
    custom_llm_provider="openai",
    messages = [{ "content": search_split_prompt,"role": "user"}],
    temperature=0.5
)
print("Response recieved")
print(response.choices[0].message.content)
arxiv_queries_list = response.choices[0].message.content.split("|")


# Each element contains vector stores for each search query developed by LLM
chunks_for_queries = []
for q in arxiv_queries_list:
    docs = ArxivLoader(query=q, load_max_docs=5).load()
    text_splitter = RecursiveCharacterTextSplitter.from_tiktoken_encoder(
        chunk_size=350, chunk_overlap=50
    )
    chunked_documents = text_splitter.split_documents(docs)
    chunks_for_queries.append(chunked_documents)

In [ ]:
print(type(chunks_for_queries))
print(len(chunks_for_queries[0]))

In [ ]:
# Instantiate the Embedding Model
embeddings = OpenAIEmbeddings(model="text-embedding-3-small",openai_api_key=apikey, base_url="https://drchat.xyz")
# Create Index- Load document chunks into the vectorstore
vectorstore_list = []
for x in chunks_for_queries:
    faiss_vectorstore = FAISS.from_documents(
        documents=x,
        embedding=embeddings,
    )
    print(type(faiss_vectorstore))
    vectorstore_list.append(faiss_vectorstore)


# Create a retriver and retrieve relevant documents for each vector store
relevant_documents_list = []
for x in vectorstore_list:
    
    relevant_documents = x.similarity_search(user_query, k = 5)
    print(type(relevant_documents))
    relevant_documents_list.append(relevant_documents)

In [ ]:
print(relevant_documents_list)

In [ ]:
question_prompt = f"""
Given the following context: {relevant_documents_list}

Answer the following question: {user_query}

Only answer the question if the answer is in the context. Otherwise, say that you don't know.
"""

response = completion(
    api_key=apikey,
    base_url="https://drchat.xyz",
    model = "gpt-3.5-turbo-16k",
    custom_llm_provider="openai",
    messages = [{ "content": question_prompt,"role": "user"}],
    temperature=0.5
)

print(response.choices[0].message.content)